<a href="https://colab.research.google.com/github/SridharSola/Tea-Dataset/blob/main/Tea_Sickness_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

#!ls "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
"""
Methods to read file names and corresponding labels and divide into train, test and validation sets. 
I choose to use 10-fold cross validation as the dataset is small. 
"""

import os #for creating and removing directories
import torch.utils.data as data
import numpy as np
from PIL import Image
import pandas as pd
import cv2
import csv
import matplotlib.pyplot as plt
import random

def my_loader(path):
  try:
    with open(path, 'rb') as f:
      img = cv2.imread(path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)#Image.fromarray(img)
      return img#Image.open(f).convert('RGB')
  except IOError:
      print('Cannot load image ' + path)

def get_class(label): #Returns the specific file names
  classes = {
      0: 'red leaf spot',
      1: 'algal leaf',
      2: 'bird eye spot',
      3: 'gray light',
      4: 'white spot',
      5: 'Anthracnose',
      6: 'brown blight',
      7: 'healthy'
  }
  return classes[label]

def Get(root): #returns the image names and labels of all images in the dataset
    dir = []
    labelList = []
    i = 0
    for i in range(0,8):
      name = get_class(i)
      file1 = root + '/' + name
      file_list = os.listdir(file1)
      for f in file_list:
        dir.append(os.path.join(file1, f))
        labelList.append(i)
    #Shuffle the image names and labels using zip 
    temp = list(zip(dir, labelList))
    random.shuffle(temp)
    t1, t2 = zip(*temp)
    imageList = list(t1)
    labelList = list(t2)
    return imageList, labelList

class ImageList(data.Dataset):
  #Actual Dataset class with necessary methods to pass to dataloader
  def __init__(self, images, labels,  transform = None, loader = my_loader):
    self.images = images
    self.labels = labels
    self.num_cls = 8
    self.transform = transform
    self.loader = loader


  def __getitem__(self, index):
    label = self.labels[index]
    imgPath = self.images[index]
    #print(imgPath)
    img = self.loader(imgPath)
    if self.transform is not None:
      img = self.transform(img)
    return img, label
  
  def __len__(self):
    return len(self.images)

  def show_img(self, index):
    #Displays the image
    image, label = self.__getitem__(index)
    image = image.permute(1, 2, 0)
    plt.axis("off")
    plt.imshow(image)
    print('Label:', label)


def get_train_test_val_lists(images, labels, K, k):
  """
  Images --> list of image names
  K --> total number of folds
  k --> current fold
  Returns list of image names for val, test, and train set
  based on kth fold which can be used to get
  objects of ImageList class again
  """
  
  num_img_pfold = int(len(images) / K) #Splits according to the current fold
  front_test = num_img_pfold*k
  end_test = num_img_pfold*(k+1)
  front_val = end_test

  
  if k == 9:
    end = len(images) - 1
    front_val = 0
  end_val = front_val + num_img_pfold
  if  k == 0:
    train_imgList = images[end_val:]
    train_labelList = labels[end_val:]
  elif k == 9:
    train_imgList = images[end_val:front_test]
    train_labelList = labels[end_val:front_test]
  else:
    train_imgList = images[0:front_test] + images[end_val:]
    train_labelList = labels[0:front_test] + labels[end_val:]
  test_imgList = images[front_test:end_test]
  test_labelList = labels[front_test:end_test]
  val_imgList = images[front_val: end_val]
  val_labelList = labels[front_val: end_val]
  
  print("Number of training images: ", len(train_imgList))
  print("Number of validation images: ", len(val_imgList))
  print("Number of test images: ", len(test_imgList))
  return train_imgList, val_imgList, test_imgList, train_labelList, val_labelList, test_labelList


In [4]:
'''

Resnet models

                          NOTE: only layers required are retained and fine-tuned.

'''
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
import torch



__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU()
        self.downsample = downsample
        self.stride = stride
        self.features_conv = self.vgg.features[:36]

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=7, end2end=True):
        self.inplanes = 64
        self.end2end = end2end
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, x):
       
        bs = x.size(0)
        f = x

        f = self.conv1(f)
        f = self.bn1(f)
        f = self.relu(f)
        f = self.maxpool(f)
        
        f = self.layer1(f)
        #print('layer1: ',f.size())
        f = self.layer2(f)
        #print('layer2: ',f.size())
        f = self.layer3(f)
        feature = f.view(bs, -1)
        #print('layer4: ',f.size())
        f = self.layer4(f)
        #print('layer4: ',f.size())

        #hook for gradcam
        #h = f.register_hook(self.activations_hook)

        f = self.avgpool(f)
        
        f = f.squeeze(3).squeeze(2)
        return f
        #return  F.normalize(f) #f

    # method for the gradient extraction
    def get_activations_gradient(self):
        return self.gradients
        
    # method for the activation exctraction
    def get_activations(self, x):
        return self.features_conv(x)

def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model




def load_resnet18(ImageNet = False, load_weights = False, path = None,  compare = True, num_classes = 8):
  if ImageNet == False:
    net = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
  else:
    net = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
  #Clipping last layer
  final_in_ftrs = net.fc.in_features
  net.fc = nn.Linear(final_in_ftrs, num_classes)
  
  return net


In [6]:
from __future__ import print_function

import argparse
import os
import shutil
import time
import random
import math

import numpy as np

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.nn.functional as F

from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd



In [ ]:
#Creating parser to store arguments to pass to main 

parser = argparse.ArgumentParser(description='Tea Sickness Dataset')

# Optimization options
parser.add_argument('--epochs', default=30, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=1, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--batch-size', default=96, type=int, metavar='N',
                    help='train batchsize')
parser.add_argument('--lr', '--learning-rate', default=0.0001, type=float,
                    metavar='LR', help='initial learning rate')
parser.add_argument('--workers', type=int, default=16,
                        help='num of workers to use')
parser.add_argument('--folds', default=10, type=int, metavar='N', help='cross validation folds')

# Checkpoints
parser.add_argument('--resume', default='', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')

#Device options
parser.add_argument('--gpu', default='0,1', type=str,
                    help='id(s) for CUDA_VISIBLE_DEVICES')

#Method options
parser.add_argument('--train-iteration', type=int, default=800,
                        help='Number of iteration per epoch')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M',  help='momentum')

parser.add_argument('--weight-decay', '--wd', default=1e-3, type=float,  metavar='W', help='weight decay (default: 1e-4)')

parser.add_argument('--print-freq', '-p', default=1000, type=int,metavar='N', help='print frequency (default: 10)')

parser.add_argument('--imagesize', type=int, default = 224, help='image size (default: 224)')


#Data
parser.add_argument('--classes', type=int, default=8)

parser.add_argument('--root', type=str, default='/content/drive/MyDrive/tea sickness dataset',
                        help="root path to train data directory")

parser.add_argument('--model-dir', default='/content/drive/MyDrive/tea sickness dataset/Checkpoint', type=str)

parser.add_argument('--logfile', default='/content/drive/MyDrive/tea sickness dataset/Tea.txt', type=str)

#args = parser.parse_args()
args = parser.parse_args(" ".split())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #To use GPU


best_acc = 0
def main(args):
    global best_acc

    #Data
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]
    imagesize = args.imagesize
    train_transform = transforms.Compose([transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.4, contrast = 0.3, saturation = 0.25, hue = 0.05),    
            transforms.Resize((args.imagesize, args.imagesize)),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ])

        
    valid_transform = transforms.Compose([transforms.ToPILImage(),
            transforms.Resize((args.imagesize,args.imagesize)),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ])
    log = open(args.logfile, 'w')
    
    #K-Fold Cross Validation Starts here
    for fold in range(0, args.folds):
      print("\n***************************************************************************************\n FOLD: ", fold)
      print("\n***************************************************************************************\n FOLD: ", fold, file =log)
      
      net = load_resnet18(False) #Creating model without loading any pretrained weights

      optimizer =  torch.optim.Adam([{"params": net.parameters(), "lr": args.lr, "momentum":args.momentum,
                                 "weight_decay":args.weight_decay}])
      lrs = []
      lrs.append(args.lr)

      if args.resume:
          print("Resuming from previous fold")
          if os.path.isfile(args.resume):
              print("=> loading checkpoint '{}'".format(args.resume))
              checkpoint = torch.load(args.resume)
              ch = checkpoint['model_state_dict']
              
              net.load_state_dict(ch)
              
              optimizer.load_state_dict(checkpoint['optimizer'])
              print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
          else:
              print("=> no checkpoint found at '{}'".format(args.resume))  

      images, labels = Get(args.root)
      train_imgs, val_imgs, test_imgs, train_lab, val_lab, test_lab = get_train_test_val_lists(images, labels, args.folds, fold) 
      train_dataset = ImageList(train_imgs, train_lab, train_transform)
      val_dataset = ImageList(val_imgs, val_lab,  valid_transform)
      test_dataset = ImageList(test_imgs, test_lab,  valid_transform)
      train_loader = torch.utils.data.DataLoader(train_dataset, args.batch_size, shuffle=True,
                                                   num_workers=args.workers, pin_memory=True)
      val_loader = torch.utils.data.DataLoader(val_dataset, args.batch_size, shuffle=False, num_workers=8)
      test_loader = torch.utils.data.DataLoader(test_dataset, args.batch_size, shuffle=False, num_workers=8)
      criterion = nn.CrossEntropyLoss().to(device)

      print("\nStarting Training\n")

      for epoch in range(args.start_epoch, args.epochs):
        if epoch == 14 or epoch == 20 or epoch == 25:
          adjust_learning_rate(optimizer, epoch)
          lrs.append(optimizer.param_groups[0]["lr"])
          print(f'Updated lr: {lrs[-1]}\n', file = log)
          print(f'Updated lr: {lrs[-1]}\n')
        train(train_loader, net, criterion, optimizer, epoch, args.epochs, log)
        acc = validate(val_loader, net, criterion, epoch)
        print("Epoch: {}   Validation Set Acc: {:.4f}".format(epoch, acc))
        print("Epoch: {}   Validation Set Acc: {:.4f}".format(epoch, acc), file = log)

        #Save best_acc and checkpoint
        is_best = acc > best_acc
        best_acc = max(acc, best_acc)
        print('\n*********************************\nBest accuracy so far is : ', '%.4f'%best_acc)
        print('\n*********************************\nBest accuracy so far is : ', '%.4f'%best_acc, log)

        save_checkpoint({'epoch': epoch+1, 'model_state_dict': net.state_dict(), 
                         'best_acc': best_acc, 'optimizer': optimizer.state_dict()},
                        is_best, 'checkpoint.pth.tar', fold)
      test(test_loader, net, criterion)
      conf_mat(net, test_loader)
def save_checkpoint(state, is_best, filename = 'checkpoint.pth.tar', fold = 0):
  epoch_num = state['epoch']
  full_bestname = os.path.join(args.model_dir, str(fold)+' model_best.pth.tar')
  if is_best:
      torch.save(state, full_bestname)

def adjust_learning_rate(optimizer, epoch):
  for param_group in optimizer.param_groups:
        param_group['lr'] /= 10
    
def accuracy(output, target, topk=(1,)):
  """Computes the precision@k for the specified values of k"""
  maxk = max(topk)
  batch_size = target.size(0)

  _, pred = output.topk(maxk, 1, True, True)
  pred = pred.t()
  correct = pred.eq(target.view(1, -1).expand_as(pred))

  res = []
  for k in topk:
      correct_k = correct[:k].view(-1).float().sum(0)
      res.append(correct_k.mul_(100.0 / batch_size))
  return res

def train(train_loader, net, criterion, optimizer, epoch, n, log):
  net.train()
  running_loss = 0.0
  correct = 0
  total=0
  for batch_idx, (data, target) in enumerate(train_loader):
    data = data.to(device)
    target = target.to(device)
    optimizer.zero_grad()
    probs = net(data)
    loss = criterion(probs, target)
    loss.backward()
    optimizer.step()
    _, preds = torch.max(probs, dim = 1)
    correct += torch.sum(preds==target).item()
    total += target.size(0)
    acc = 100 * correct/total
    if batch_idx%args.print_freq == 0:
      print("Training Epoch: {}/{}\tLoss: {:.4f}\nTrain Accuracy: {:.4f}". format(epoch, n, loss.item(), acc))
      print('Training Epoch: {}/{}\tLoss: {:.4f}\nTrain Accuracy: {:.4f}' . format(epoch, n, loss.item(), acc), file = log)

def validate(val_loader, net, criterion, epoch):
  net.eval()
  batch_loss = 0
  total=0
  correct=0
  with torch.no_grad():
    for batch_idx, (data, target) in enumerate(val_loader):
      data =  data.to(device)
      target = target.to(device)
      probs = net(data)
      loss = criterion(probs, target)
      _, preds = torch.max(probs, dim = 1)
      correct += torch.sum(preds==target).item()
      total += target.size(0)
  acc = 100 * correct/total
  #print(f"Validation Set Accuracy: {(100 * correct/total):.4f}\n")
  return acc

def test(test_loader, net, criterion):
  net.eval()
  batch_loss = 0
  total=0
  correct=0
  with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
      data =  data.to(device)
      target = target.to(device)
      probs = net(data)
      loss = criterion(probs, target)
      _, preds = torch.max(probs, dim = 1)
      correct += torch.sum(preds==target).item()
      total += target.size(0)
  acc = 100 * correct/total
  #print(f"Validation Set Accuracy: {(100 * correct/total):.4f}\n")
  print("Test accuracy: ", acc)

def conf_mat(net, test):
  y_pred = []
  y_true = []
  net.eval()
  # iterate over test data
  for inputs, labels in test:
          output = net(inputs) # Feed Network

          output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
          y_pred.extend(output) # Save Prediction
          
          labels = labels.data.cpu().numpy()
          y_true.extend(labels) # Save Truth

  # constant for classes
  classes = {
      'red leaf spot',
      'algal leaf',
      'bird eye spot',
      'gray light',
      'white spot',
      'Anthracnose',
      'brown blight',
      'healthy'
  }

  # Build confusion matrix
  cf_matrix = confusion_matrix(y_true, y_pred)
  df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
                      columns = [i for i in classes])

  df_conf_norm = df_cm / df_cm.sum(axis=1)
  plt.figure(figsize = (12,7))
  sn.heatmap(df_conf_norm, annot=True)
  plt.savefig('output.png')

if __name__ == "__main__":
    
    main(args)
    print("Completed K-fold Cross Validation!")                
                  
                  
                  
